### Doc 360 Call Activity

In [112]:
import polars as pl
import gc
import pandas as pd
from datetime import datetime, timedelta,date
import json

In [113]:
# load variables from JSON
with open('vars_wk.json', 'r') as json_file:
    js = json.load(json_file)
roster_file = js['roster_file']
num_weeks_calls = js['num_weeks_calls']
quarter_start = datetime.strptime(js['quarter_start'], '%Y-%m-%d').date()
bucket = js['bucket']


dflib = f's3://{bucket}/BIT/dataframes/'
rst = f's3://{bucket}/BIT/roster/'

In [114]:
# Utility Functions -
def load(df, lib=dflib):
    globals()[df] = pl.read_parquet(f'{lib}{df}.parquet')

In [115]:
# Importing Dependencies
load('temp_calls')
load('roster')
load('temp_abbv')
load('MASTER_UNI')
#fixes for vortex import -> Probably caused by Polars Upgrades
temp_calls = temp_calls.with_columns(pl.col('SalesRepIID').cast(pl.Int64))
temp_abbv = temp_abbv.with_columns(pl.col('SalesRepIID').cast(pl.Int64))

roster2 = (
    pl.read_parquet(f'{rst}MasterRoster_{roster_file}.parquet')
    .with_columns(
        pl.col('SalesRepIID').cast(pl.Int64),
        Rep = pl.col('EmpFName')+' '+pl.col('EmpLName')
    ).select(['SalesRepIID','Rep'])
)

In [116]:
# Processing - 
# IW CALLS - 
iw_calls = (
    temp_calls
    .filter((pl.col('call_week')<=num_weeks_calls)).filter(pl.col('CallDate')>= quarter_start)
    # .join(MASTER_UNI.select(['IID','Territory']),left_on = 'AttendeeIID', right_on = 'IID',how='left')
    # .join(roster, on = 'SalesRepIID' , how = 'left')
    # .filter(pl.col('Territory')==pl.col('GEO'))
    # .drop(['Territory','GEO'])
)

# ABBV CALLS - 
ab_calls = (
    temp_abbv
    .filter((pl.col('call_week')<=num_weeks_calls)).filter(pl.col('CallDate')>= quarter_start)
)

# Only Keeping Abbv Calls for those HCPs who were also called in iw_calls - 
# ab_calls = (
#     ab_calls.join(
#         iw_calls.select(['AttendeeIID']).unique(),
#         on = ['AttendeeIID'],
#         how = 'inner'
#     )
# )

# m2-
# ab_calls = ab_calls.filter(
#     pl.col('AttendeeIID').is_in(
#         list(iw_calls.select(['AttendeeIID']).unique())
#     )
# )

calls = (
    iw_calls.vstack(ab_calls)
    .with_columns(
        pl.when(pl.col('source').is_in(['FRX','AEM'])).then(pl.lit(5)).otherwise(pl.lit(1)).alias('Salesforce_id')
    )
)

In [117]:
temp1 = (
    calls
    .select(['CallID','AttendeeIID','CallDate','CallType','SalesRepIID','Salesforce_id'])
    .join(roster2,on='SalesRepIID',how='left')
    .drop('SalesRepIID')
    .with_columns(
        pl.col('CallID').cast(pl.Int64),
        pl.lit('LINZESS').alias('P1'),
        pl.lit('\\N').alias('P2'),
        pl.lit('\\N').alias('P3'),
        pl.col('CallDate').cast(pl.Date)
    )
    .select('CallID','AttendeeIID','Salesforce_id','CallDate','CallType','Rep','P1','P2','P3')
)
temp1.columns = ['CallId','DoctorId','Salesforce_id','Date','CallType','Rep','P1','P2','P3']
temp1 = temp1.unique()

In [118]:
#Exporting Feeds-
OUT = 's3://vortex-staging-a65ced90/BIT/output/Doc360/'
temp1.to_pandas().to_csv(f'{OUT}Doc360_CallActivity_Feed.txt', sep='|',lineterminator='\r\n',index=False)
print('Doc360_CallActivity_Feed Exported !')

Doc360_CallActivity_Feed Exported !


---